# Pipeline de Análise de Risco de Mercado

In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
from scipy.stats import norm
import plotly.express as px
import plotly.graph_objects as go

## 1. Coleta de Dados Históricos

Coletamos os dados históricos de dois ativos: a ação da Petrobras (PETR4.SA) e o índice Ibovespa (^BVSP), que servirá como benchmark. A coleta é feita para os últimos 6 meses.

In [2]:
# Definindo os tickers e o período
tickers = ['PETR4.SA', '^BVSP']
end_date = pd.Timestamp.now()
start_date = end_date - pd.DateOffset(months=6)

# Baixando os dados de fechamento, que já são ajustados por padrão
precos = yf.download(tickers, start=start_date, end=end_date)['Close']
precos = precos.dropna() # Garante que não há dias sem negociação para ambos os ativos

print("Amostra dos preços dos ativos:")
precos.head()

/tmp/ipykernel_1091/422196918.py:7: FutureWarning: YF.download() has changed argument auto_adjust default to True
  precos = yf.download(tickers, start=start_date, end=end_date)['Close']
[*********************100%***********************]  2 of 2 completed

Amostra dos preços dos ativos:


Ticker,PETR4.SA,^BVSP
Date,,
2025-02-03,35.524513,125970.0
2025-02-04,35.174004,125147.0
2025-02-05,34.918224,125534.0
2025-02-06,34.851913,126225.0
2025-02-07,34.652977,124619.0


## 2. Cálculo dos Indicadores de Risco

### 2.1. Retornos Logarítmicos

In [3]:
# Calculando os retornos diários
retornos = np.log(precos / precos.shift(1)).dropna()

print("Amostra dos retornos diários:")
retornos.head()

Amostra dos retornos diários:


Ticker,PETR4.SA,^BVSP
Date,,
2025-02-04,-0.009916,-0.006555
2025-02-05,-0.007298,0.003088
2025-02-06,-0.001901,0.005489
2025-02-07,-0.005724,-0.012805
2025-02-10,0.006811,0.007618


### 2.2. Volatilidade, VaR e Correlação

In [4]:
# Volatilidade anualizada
volatilidade_anualizada = retornos.std() * np.sqrt(252)

# VaR Paramétrico (95% de confiança)
confianca = 0.95
media_retornos = retornos.mean()
desvio_padrao_retornos = retornos.std()
z_score = norm.ppf(1 - confianca)
var_parametrico = -(media_retornos + z_score * desvio_padrao_retornos)

# Correlação entre os ativos
correlacao = retornos.corr()

## 3. Apresentação e Visualização dos Resultados

*(Esta seção será desenvolvida no próximo commit)*

In [5]:
# Tabela de Indicadores de Risco
fig_tabela = go.Figure(data=[go.Table(
    header=dict(values=['Indicador', 'PETR4.SA', '^BVSP']),
    cells=dict(values=[
        ['Volatilidade Anualizada', 'VaR Diário (95%)'],
        [f"{volatilidade_anualizada['PETR4.SA']:.2%}", f"{var_parametrico['PETR4.SA']:.2%}"],
        [f"{volatilidade_anualizada['^BVSP']:.2%}", f"{var_parametrico['^BVSP']:.2%}"]
    ]))
])
fig_tabela.update_layout(title_text='Tabela de Indicadores de Risco')
fig_tabela.show()

In [6]:
# Gráfico da série histórica de preços
fig_precos = px.line(precos, title='Série Histórica de Preços (Últimos 6 Meses)')
fig_precos.show()

In [7]:
# Gráfico dos retornos diários
fig_retornos = px.line(retornos, title='Retornos Diários dos Ativos')
fig_retornos.show()

In [8]:
# Heatmap da Matriz de Correlação
fig_heatmap = px.imshow(correlacao, text_auto=True, title='Heatmap de Correlação entre os Ativos')
fig_heatmap.show()

In [9]:
# Histograma dos retornos
fig_hist = px.histogram(retornos, x=['PETR4.SA', '^BVSP'], marginal='box', title='Distribuição dos Retornos Diários')
fig_hist.show()